In [1]:


import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context
    
    


# 1. Download and Explore Dataset

In [2]:
#download all the dependencies that we will need.

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy

!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

#print('Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
import pandas as pd 

import requests 

from bs4 import BeautifulSoup 

headers = { 'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36", } 
data = requests.request("GET", "https://en.wikipedia.org/wiki/Category:Suburbs_of_Sydney", headers=headers, verify=False).text

soup = BeautifulSoup(data, 'html.parser')

neighborhoodList = []

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
# create a new DataFrame from the list
Sydney_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Sydney_df.head()

C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,Neighborhood
0,"► Alexandria, New South Wales‎ (1 C, 9 P)"
1,"► Annandale, New South Wales‎ (1 C, 11 P)"
2,"► Arncliffe, New South Wales‎ (6 P)"
3,"► Artarmon, New South Wales‎ (5 P)"
4,"► Ashfield, New South Wales‎ (1 C, 3 P)"


In [4]:
!pip install geocoder
import geocoder

# add latitude and longitude to Neighbourhoods Table
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Sydney, Australia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Sydney_df["Neighborhood"].tolist() ]
#coords

In [5]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


# merge the coordinates into the original dataframe
Sydney_df['Latitude'] = df_coords['Latitude']
Sydney_df['Longitude'] = df_coords['Longitude']


import folium

latitude = -33.8696
longitude = 151.20695

map_Sydney = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(Sydney_df['Latitude'], Sydney_df['Longitude'], Sydney_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Sydney)  
    
map_Sydney

In [6]:
# save the map as HTML file
map_Sydney.save('map_Sydney.html')

# 2. Explore Neighborhoods in Sydney

In [7]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2' # your Foursquare ID
CLIENT_SECRET = 'TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2
CLIENT_SECRET:TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV


In [8]:
Sydney_df.loc[0, 'Latitude']

-33.91236999999995

In [9]:
#Get the neighborhood's latitude and longitude values.

neighborhood_latitude = Sydney_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Sydney_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Sydney_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of ►  Alexandria, New South Wales‎ (1 C, 9 P) are -33.91236999999995, 151.19703000000004.


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2', 
            'TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV', 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()['response']['groups'][0]['items']
        results=requests.request("GET", url, headers=headers, verify=False).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
#Now write the code to run the above function on each neighborhood and create a new dataframe called NYork_venues.

Sydney_venues = getNearbyVenues(names=Sydney_df['Neighborhood'],
                                   latitudes=Sydney_df['Latitude'],
                                   longitudes=Sydney_df['Longitude'],
                                   radius=500
                                  )

►  Alexandria, New South Wales‎ (1 C, 9 P)


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


►  Annandale, New South Wales‎ (1 C, 11 P)


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


►  Arncliffe, New South Wales‎ (6 P)


...

C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


►  Woolloomooloo‎ (6 P)


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


►  Woolwich, New South Wales‎ (5 P)


C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [12]:
print(Sydney_venues.shape)
Sydney_venues.head()

(3872, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"► Alexandria, New South Wales‎ (1 C, 9 P)",-33.91237,151.19703,The Potting Shed at The Grounds,-33.911094,151.194306,Bar
1,"► Alexandria, New South Wales‎ (1 C, 9 P)",-33.91237,151.19703,The Grounds of Alexandria,-33.910774,151.194406,Café
2,"► Alexandria, New South Wales‎ (1 C, 9 P)",-33.91237,151.19703,L'Americano,-33.910241,151.198653,Café
3,"► Alexandria, New South Wales‎ (1 C, 9 P)",-33.91237,151.19703,Alexandria Basketball Stadium,-33.908358,151.195578,Basketball Stadium
4,"► Alexandria, New South Wales‎ (1 C, 9 P)",-33.91237,151.19703,Al Aseel,-33.912632,151.193462,Lebanese Restaurant


In [13]:
Sydney_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"► Alexandria, New South Wales‎ (1 C, 9 P)",22,22,22,22,22,22
"► Annandale, New South Wales‎ (1 C, 11 P)",19,19,19,19,19,19
"► Arncliffe, New South Wales‎ (6 P)",12,12,12,12,12,12
"► Artarmon, New South Wales‎ (5 P)",16,16,16,16,16,16
"► Ashfield, New South Wales‎ (1 C, 3 P)",30,30,30,30,30,30
"► Asquith, New South Wales‎ (7 P)",4,4,4,4,4,4
"► Auburn, New South Wales‎ (8 P)",11,11,11,11,11,11
"► Audley, New South Wales‎ (5 P)",32,32,32,32,32,32
"► Avalon Beach, New South Wales‎ (6 P)",11,11,11,11,11,11


In [14]:
print('There are {} uniques categories.'.format(len(Sydney_venues['Venue Category'].unique())))

There are 299 uniques categories.


# 3. Analyze Each Neighborhood

In [15]:
# one hot encoding
Sydney_onehot = pd.get_dummies(Sydney_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Sydney_onehot['Neighborhood'] = Sydney_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Sydney_onehot.columns[-1]] + list(Sydney_onehot.columns[:-1])
Sydney_onehot = Sydney_onehot[fixed_columns]

Sydney_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bay,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat Launch,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cambodian Restaurant,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Cha Chaan Teng,Chaat Place,Cheese Shop,Chinese Restaurant,Churrascaria,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cruise Ship,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Newsstand,Noodle House,Office,Organic Grocery,Outdoor Supply Store,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Roast Chicken Joint,Pet Store,Pharmacy,Picnic Area,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tour

In [16]:
Sydney_onehot.shape

(3872, 300)

In [17]:
Sydney_grouped = Sydney_onehot.groupby('Neighborhood').mean().reset_index()
Sydney_grouped

,Neighborhood,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bay,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat Launch,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cambodian Restaurant,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Cha Chaan Teng,Chaat Place,Cheese Shop,Chinese Restaurant,Churrascaria,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cruise Ship,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean BBQ Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Newsstand,Noodle House,Office,Organic Grocery,Outdoor Supply Store,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Roast Chicken Joint,Pet Store,Pharmacy,Picnic Area,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Syrian Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tour

In [18]:
Sydney_grouped.shape

(159, 300)

In [191]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 3

for hood in Sydney_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Sydney_grouped[Sydney_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----►  Alexandria, New South Wales‎ (1 C, 9 P)----
               venue  freq
0               Café  0.27
1     Sandwich Place  0.05
2  Electronics Store  0.05


----►  Annandale, New South Wales‎ (1 C, 11 P)----
           venue  freq
0           Café  0.32
1           Park  0.11
2  Grocery Store  0.11


----►  Arncliffe, New South Wales‎ (6 P)----
                 venue  freq
0             Platform  0.17
1  Rental Car Location  0.08
2      Thai Restaurant  0.08


----►  Artarmon, New South Wales‎ (5 P)----
                 venue  freq
0  Japanese Restaurant  0.12
1                 Café  0.12
2     Ramen Restaurant  0.06


----►  Ashfield, New South Wales‎ (1 C, 3 P)----
                 venue  freq
0  Dumpling Restaurant  0.10
1                 Café  0.10
2  Japanese Restaurant  0.07


----►  Asquith, New South Wales‎ (7 P)----
           venue  freq
0    Supermarket  0.25
1  Train Station  0.25
2     Hobby Shop  0.25


----►  Auburn, New South Wales‎ (8 P)----
                 venue 

2  Thai Restaurant  0.08


----►  Kurnell, New South Wales‎ (4 P)----
                    venue  freq
0  Furniture / Home Store   0.2
1                    Café   0.2
2         Nature Preserve   0.2


----►  Kyeemagh, New South Wales‎ (5 P)----
         venue  freq
0        Beach  0.25
1         Park  0.12
2  Sports Club  0.12


----►  La Perouse, New South Wales‎ (10 P)----
           venue  freq
0          Beach  0.23
1           Café  0.15
2  Historic Site  0.08


----►  Lane Cove, New South Wales‎ (8 P)----
         venue  freq
0         Café   0.1
1  Pizza Place   0.1
2       Bakery   0.1


----►  Lavender Bay‎ (6 P)----
         venue  freq
0         Café  0.30
1  Coffee Shop  0.05
2    Juice Bar  0.05


----►  Leumeah, New South Wales‎ (4 P)----
           venue  freq
0  Train Station  0.14
1  Grocery Store  0.14
2   Tennis Court  0.14


----►  Lewisham, New South Wales‎ (7 P)----
                venue  freq
0                Café  0.16
1  Light Rail Station  0.11
2         Music 

                   venue  freq
0                   Café  0.17
1                 Bakery  0.07
2  Vietnamese Restaurant  0.07


----►  Waverton, New South Wales‎ (4 P)----
             venue  freq
0             Café  0.25
1             Park  0.17
2  Thai Restaurant  0.08


----►  Werrington, New South Wales‎ (1 C, 5 P)----
                venue  freq
0                 Bar   1.0
1  African Restaurant   0.0
2                Park   0.0


----►  West Ryde, New South Wales‎ (4 P)----
                 venue  freq
0          Pizza Place  0.12
1  Japanese Restaurant  0.08
2          Supermarket  0.08


----►  Wilberforce, New South Wales‎ (7 P)----
                venue  freq
0        Soccer Field   1.0
1  African Restaurant   0.0
2            Pharmacy   0.0


----►  Willoughby, New South Wales‎ (5 P)----
                venue  freq
0                Café   1.0
1  African Restaurant   0.0
2            Pharmacy   0.0


----►  Windsor, New South Wales‎ (30 P)----
              venue  freq
0        

In [192]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [193]:
import numpy as np #

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Sydney_grouped['Neighborhood']

for ind in np.arange(Sydney_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Sydney_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"► Alexandria, New South Wales‎ (1 C, 9 P)",Café,Sandwich Place,Electronics Store,Miscellaneous Shop,Clothing Store
1,"► Annandale, New South Wales‎ (1 C, 11 P)",Café,Park,Grocery Store,Fish & Chips Shop,Snack Place
2,"► Arncliffe, New South Wales‎ (6 P)",Platform,Rental Car Location,Thai Restaurant,Gym,Grocery Store
3,"► Artarmon, New South Wales‎ (5 P)",Japanese Restaurant,Café,Ramen Restaurant,Bakery,Convenience Store
4,"► Ashfield, New South Wales‎ (1 C, 3 P)",Dumpling Restaurant,Café,Japanese Restaurant,Chinese Restaurant,Platform


# 4. Cluster Neighborhoods

In [194]:
#import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Sydney_grouped_clustering = Sydney_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Sydney_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 1, 1, 1, 1, 0, 0, 0])

In [195]:
kmeans.labels_

array([0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 3, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 3, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 3, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 3, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 4, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 3, 1, 1, 1])

In [196]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.pop('Cluster Labels')
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Sydney_merged = Sydney_df

# merge NYork_grouped with Sydney_data to add latitude/longitude for each neighborhood
Sydney_merged = Sydney_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

Sydney_merged.head() # check the last columns!

KeyError: 'Cluster Labels'

In [197]:

import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Sydney_merged['Latitude'], Sydney_merged['Longitude'], Sydney_merged['Neighborhood'], Sydney_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [198]:
# save the map as HTML file
map_clusters.save('map_Sydneyclusters.html')

# 5. Examine Clusters

In [199]:
#Cluster 1

Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 0, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"► Alexandria, New South Wales‎ (1 C, 9 P)",Café,Sandwich Place,Electronics Store,Miscellaneous Shop,Clothing Store
1,"► Annandale, New South Wales‎ (1 C, 11 P)",Café,Park,Grocery Store,Fish & Chips Shop,Snack Place
7,"► Audley, New South Wales‎ (5 P)",Café,Portuguese Restaurant,Bus Stop,Pub,Platform
8,"► Avalon Beach, New South Wales‎ (6 P)",Café,Japanese Restaurant,Beach,Coffee Shop,Bakery
9,"► Balmain East, New South Wales‎ (5 P)",Café,Italian Restaurant,Gastropub,Bowling Green,Pier
10,"► Balmain, New South Wales‎ (2 C, 37 P)",Café,Pub,Japanese Restaurant,Sushi Restaurant,Bar
11,"► Bankstown, New South Wales‎ (1 C, 21 P)",Café,Buffet,Fast Food Restaurant,Sports Bar,Multiplex
12,"► Bargo, New South Wales‎ (6 P)",Café,Boat Launch,Clothing Store,Trail,Pastry Shop
15,"► Bellevue Hill, New South Wales‎ (1 C, 3 P)",Café,Yoga Studio,Liquor Store,Pastry Shop,Peruvian Roast Chicken Joint
16,"► Belrose, New South Wales‎ (4 P)",Health & Beauty Service,Café,Grocery Store,Print Shop,Peruvian Roast Chicken Joint


In [200]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 3].shape

(5, 9)

In [201]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 1, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,"► Arncliffe, New South Wales‎ (6 P)",Platform,Rental Car Location,Thai Restaurant,Gym,Grocery Store
3,"► Artarmon, New South Wales‎ (5 P)",Japanese Restaurant,Café,Ramen Restaurant,Bakery,Convenience Store
4,"► Ashfield, New South Wales‎ (1 C, 3 P)",Dumpling Restaurant,Café,Japanese Restaurant,Chinese Restaurant,Platform
5,"► Asquith, New South Wales‎ (7 P)",Supermarket,Train Station,Hobby Shop,Fast Food Restaurant,African Restaurant
6,"► Auburn, New South Wales‎ (8 P)",Coffee Shop,Lebanese Restaurant,Noodle House,Gym,Supermarket
13,"► Baulkham Hills, New South Wales‎ (4 P)",Supermarket,Convenience Store,Café,Shopping Mall,Seafood Restaurant
17,"► Birchgrove, New South Wales‎ (11 P)",Convenience Store,Athletics & Sports,African Restaurant,Pastry Shop,Pharmacy
19,"► Bondi Junction, New South Wales‎ (5 P)",Japanese Restaurant,Café,Burger Joint,Coffee Shop,Gym
20,"► Bondi, New South Wales‎ (18 P)",Dog Run,Thai Restaurant,Sports Club,Park,Paper / Office Supplies Store
24,"► Burwood, New South Wales‎ (13 P)",Café,Chinese Restaurant,Fast Food Restaurant,Supermarket,Asian Restaurant


In [202]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 2, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
149,"► Warragamba, New South Wales‎ (8 P)",Food & Drink Shop,African Restaurant,Pharmacy,Peruvian Roast Chicken Joint,Persian Restaurant


In [203]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 3, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
27,► Cammeray‎ (4 P),Park,Japanese Restaurant,Café,African Restaurant,Peruvian Roast Chicken Joint
57,"► Galston, New South Wales‎ (4 P)",Water Park,Park,African Restaurant,Peruvian Roast Chicken Joint,Persian Restaurant
93,"► Marsfield, New South Wales‎ (5 P)",Park,Pizza Place,African Restaurant,Peruvian Roast Chicken Joint,Persian Restaurant
106,"► North Parramatta, New South Wales‎ (7 P)",Park,Music Store,Grocery Store,Peruvian Roast Chicken Joint,Persian Restaurant
159,► Wollstonecraft‎ (6 P),Peruvian Roast Chicken Joint,Park,African Restaurant,Pharmacy,Persian Restaurant


In [204]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 4, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
130,"► Rouse Hill, New South Wales‎ (6 P)",Bus Stop,African Restaurant,Park,Peruvian Roast Chicken Joint,Persian Restaurant


In [205]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 5, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


In [189]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 6, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


In [190]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 7, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


In [124]:
Sydney_merged.loc[Sydney_merged['Cluster Labels'] == 8, Sydney_merged.columns[[0] + list(range(4, Sydney_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
